![Illustration of silhouetted heads](mentalhealth.jpg)

Does going to university in a different country affect your mental health? A Japanese international university surveyed its students in 2018 and published a study the following year that was approved by several ethical and regulatory boards.

The study found that international students have a higher risk of mental health difficulties than the general population, and that social connectedness (belonging to a social group) and acculturative stress (stress associated with joining a new culture) are predictive of depression.


Explore the `students` data using PostgreSQL to find out if you would come to a similar conclusion for international students and see if the length of stay is a contributing factor.

Here is a data description of the columns you may find helpful.

| Field Name    | Description                                      |
| ------------- | ------------------------------------------------ |
| `inter_dom`     | Types of students (international or domestic)   |
| `japanese_cate` | Japanese language proficiency                    |
| `english_cate`  | English language proficiency                     |
| `academic`      | Current academic level (undergraduate or graduate) |
| `age`           | Current age of student                           |
| `stay`          | Current length of stay in years                  |
| `todep`         | Total score of depression (PHQ-9 test)           |
| `tosc`          | Total score of social connectedness (SCS test)   |
| `toas`          | Total score of acculturative stress (ASISS test) |

In [2]:
-- Run this code to view the data in students
SELECT * 
FROM students;

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,Inter,SEA,Male,Grad,24.0,4.0,5.0,Long,3.0,Average,5.0,High,,Yes,No,No,No,0.0,Min,34.0,23.0,9.0,11.0,8.0,11.0,2.0,27.0,91.0,5.0,5.0,6.0,3.0,2.0,1.0,4.0,1.0,3.0,4.0,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
1,Inter,SEA,Male,Grad,28.0,5.0,1.0,Short,4.0,High,4.0,High,,No,No,No,No,2.0,Min,48.0,8.0,7.0,5.0,4.0,3.0,2.0,10.0,39.0,7.0,7.0,7.0,4.0,4.0,4.0,4.0,1.0,1.0,1.0,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
2,Inter,SEA,Male,Grad,25.0,4.0,6.0,Long,4.0,High,4.0,High,Yes,Yes,No,No,No,2.0,Min,41.0,13.0,4.0,7.0,6.0,4.0,3.0,14.0,51.0,3.0,3.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,NaN,No,No,No,No,No,No,No,No,No,No,No
3,Inter,EA,Female,Grad,29.0,5.0,1.0,Short,2.0,Low,3.0,Average,No,No,No,No,No,3.0,Min,37.0,16.0,10.0,10.0,8.0,6.0,4.0,21.0,75.0,5.0,5.0,5.0,5.0,5.0,2.0,2.0,2.0,4.0,4.0,NaN,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No
4,Inter,EA,Female,Grad,28.0,5.0,1.0,Short,1.0,Low,3.0,Average,Yes,No,No,No,No,3.0,Min,37.0,15.0,12.0,5.0,8.0,7.0,4.0,31.0,82.0,5.0,5.0,5.0,2.0,5.0,2.0,5.0,5.0,4.0,4.0,NaN,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,140,,,,,,,,,
282,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137,131,,,,,,,,,
283,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,202,,,,,,,,,
284,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,207,,,,,,,,,


In [ ]:
-- Start coding here...

Explaratory Data Analysis
Finding Null Values

In [3]:
SELECT * FROM students
WHERE todep IS NULL OR tosc IS NULL OR toas IS NULL OR inter_dom IS NULL;


,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,222.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,249.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,203.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,247.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.0,42.0,NaN,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,128.0,137.0,66.0,61.0,30.0,46.0,19.0,65.0,21.0,45.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,172.0,54.0,NaN,107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.0,140.0,131.0,202.0,207.0,238.0,222.0,249.0,203.0,247.0,223.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,172.0,NaN,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,128.0,137.0,66.0,61.0,30.0,46.0,19.0,65.0,21.0,45.0


In [8]:
SELECT 
    COUNT(*) AS total_rows,
    SUM(CASE WHEN todep IS NULL THEN 1 ELSE 0 END) AS null_todep,
    SUM(CASE WHEN tosc IS NULL THEN 1 ELSE 0 END) AS null_tosc,
    SUM(CASE WHEN toas IS NULL THEN 1 ELSE 0 END) AS null_toas
FROM students;


,total_rows,null_todep,null_tosc,null_toas
0,286,18,18,18


In [11]:
SELECT COUNT(*) AS international_students
FROM students
WHERE inter_dom = 'Inter';


,international_students
0,201


1. Do International Students Have Higher Mental Health Difficulties?
Comparing the mental health outcomes (PHQ-9, SCS, and ASISS) between international and domestic students.

In [19]:
SELECT inter_dom, 
       ROUND(AVG(todep), 2) AS avg_phq9, 
       ROUND(AVG(tosc), 2) AS avg_scs, 
       ROUND(AVG(toas), 2) AS avg_asiss
FROM students
GROUP BY inter_dom;


,inter_dom,avg_phq9,avg_scs,avg_asiss
0,Dom,8.61,37.64,62.84
1,,NaN,NaN,NaN
2,Inter,8.04,37.42,75.56


International students experience significantly higher acculturative stress (75.56) compared to domestic students (62.84), though their depression levels are similar, with international students scoring slightly lower (8.04 vs. 8.61). Both groups have comparable social connectedness scores, suggesting similar levels of social integration. The elevated acculturative stress among international students highlights the unique challenges they face adjusting to a new cultural environment, even if their depression levels aren't notably higher.

2. Is Social Connectedness Predictive of Depression?
Exploring the relationship between social connectedness (SCS) and depression (PHQ-9) scores to see if students with lower social connectedness tend to have higher depression scores.

In [20]:
SELECT ROUND(CAST(CORR(todep, tosc) AS NUMERIC), 2) AS phq_scs_corr
FROM students
WHERE inter_dom = 'Inter'
AND todep IS NOT NULL
AND tosc IS NOT NULL;

,phq_scs_corr
0,-0.54


The correlation coefficient of -0.54 between PHQ-9 (depression) and SCS (social connectedness) indicates a moderate negative relationship. This suggests that as social connectedness increases, depression tends to decrease among international students, highlighting the importance of social support in reducing depression levels.

3. Is Acculturative Stress Predictive of Depression?
Exploring whether acculturative stress (ASISS) is associated with higher depression among international students.

In [21]:
SELECT ROUND(CAST(CORR(todep, toas) AS NUMERIC), 2) AS phq_asiss_corr
FROM students
WHERE inter_dom = 'Inter'
AND todep IS NOT NULL
AND toas IS NOT NULL;

,phq_asiss_corr
0,0.41


The correlation coefficient of 0.41 between PHQ-9 (depression) and ASISS (acculturative stress) indicates a moderate positive relationship. This suggests that higher levels of acculturative stress are associated with increased depression among international students, indicating that adjusting to a new culture contributes to their mental health challenges.

4. Does the Length of Stay Impact Mental Health?
Examining whether the length of stay (in years) affects mental health outcomes (PHQ-9, SCS, and ASISS).

In [22]:
SELECT stay, 
       ROUND(AVG(todep), 2) AS avg_phq9, 
       ROUND(AVG(tosc), 2) AS avg_scs, 
       ROUND(AVG(toas), 2) AS avg_asiss
FROM students
WHERE inter_dom = 'Inter'
AND todep IS NOT NULL
AND tosc IS NOT NULL
AND toas IS NOT NULL
GROUP BY stay
ORDER BY stay;


,stay,avg_phq9,avg_scs,avg_asiss
0,1,7.48,38.11,72.80
1,2,8.28,37.08,77.67
2,3,9.09,37.13,78.00
3,4,8.57,33.93,87.71
4,5,0.00,34.00,91.00
5,6,6.00,38.00,58.67
6,7,4.00,48.00,45.00
7,8,10.00,44.00,65.00
8,10,13.00,32.00,50.00


The data shows that depression (PHQ-9) scores generally increase with length of stay, peaking at 13 for those who have stayed for 10 years, indicating worsening mental health over time for some students. Social connectedness (SCS) fluctuates slightly, but decreases significantly for longer stays, particularly dropping to 32 for students staying 10 years, suggesting a decline in social integration over time. Acculturative stress (ASISS) tends to rise with longer stays, peaking at 91 for those who have stayed 5 years, but then decreases for stays of 7 to 10 years, possibly indicating eventual adaptation to the new culture after prolonged exposure.

5. Do Language Proficiency Levels Affect Mental Health?
Since language proficiency (both in Japanese and English) could affect students' ability to integrate and cope, it’s worth checking if proficiency levels impact mental health outcomes.

Japanese Language Proficiency:

In [17]:
SELECT japanese_cate, 
       ROUND(AVG(todep), 2) AS avg_phq9, 
       ROUND(AVG(tosc), 2) AS avg_scs, 
       ROUND(AVG(toas), 2) AS avg_asiss
FROM students
WHERE inter_dom = 'Inter'
GROUP BY japanese_cate;


,japanese_cate,avg_phq9,avg_scs,avg_asiss
0,Average,8.38,37.56,75.45
1,High,7.40,38.48,72.00
2,Low,7.91,36.99,76.65


Students with average Japanese language proficiency have the highest depression (PHQ-9) scores at 8.38, while those with high proficiency report the lowest at 7.40, suggesting that better language skills may correlate with lower levels of depression. Additionally, social connectedness (SCS) remains relatively consistent across proficiency levels, but students with high Japanese proficiency show slightly better scores, indicating that language skills might contribute to a sense of belonging and lower acculturative stress (ASISS).

English Language Proficiency:

In [18]:
SELECT english_cate, 
       ROUND(AVG(todep), 2) AS avg_phq9, 
       ROUND(AVG(tosc), 2) AS avg_scs, 
       ROUND(AVG(toas), 2) AS avg_asiss
FROM students
WHERE inter_dom = 'Inter'
GROUP BY english_cate;


,english_cate,avg_phq9,avg_scs,avg_asiss
0,Average,8.46,36.93,75.66
1,High,7.93,37.42,75.50
2,Low,8.00,40.29,76.43


Students with average English language proficiency report the highest depression (PHQ-9) score of 8.46, while those with high proficiency have a lower average score of 7.93, indicating that better English skills may be associated with reduced depression levels. Additionally, students with low English proficiency have the highest social connectedness (SCS) score at 40.29, suggesting that even with lower language skills, they may still feel a sense of belonging, while their acculturative stress (ASISS) scores remain relatively stable across proficiency levels.

Additional Areas to Explore
Control for Other Factors:

Exploring if age or academic level (undergraduate vs. graduate) to see if these factors further influence mental health outcomes for international students.


In [23]:
SELECT academic, age, 
       ROUND(AVG(todep), 2) AS avg_phq9
FROM students
WHERE inter_dom = 'Inter'
GROUP BY academic, age;


,academic,age,avg_phq9
0,Grad,25,6.33
1,Under,29,8.00
2,Grad,31,5.75
3,Under,17,4.67
4,Grad,23,3.00
5,Grad,24,3.00
6,Under,23,11.00
7,Grad,29,2.33
8,Under,21,9.23
9,Under,19,8.44


The data shows that undergraduate students tend to have higher average depression (PHQ-9) scores compared to graduate students, with scores ranging from 4.67 for those under 17 to 9.23 for those under 21. In contrast, graduate students display generally lower depression scores, with the highest average being 6.33 for those under 25, indicating that older students and those in graduate programs may experience less depression on average. Additionally, the scores highlight variability in depression levels among younger students, suggesting that age and academic level play a significant role in mental health outcomes.

Does Support from Specific Social Groups Affect Mental Health?
Exploring how the presence of certain support networks (e.g., partner, friends, parents) affects mental health scores.

In [40]:
SELECT partner_bi, friends_bi, parents_bi, relative_bi, 
       ROUND(AVG(todep), 2) AS avg_phq9, 
       ROUND(AVG(tosc), 2) AS avg_scs, 
       ROUND(AVG(toas), 2) AS avg_asiss
FROM students
WHERE inter_dom = 'Inter'
GROUP BY partner_bi, friends_bi, parents_bi, relative_bi;

,partner_bi,friends_bi,parents_bi,relative_bi,avg_phq9,avg_scs,avg_asiss
0,Yes,No,Yes,No,8.54,34.15,73.77
1,No,No,Yes,Yes,4.40,41.60,73.80
2,No,Yes,Yes,No,7.60,37.00,79.20
3,Yes,Yes,Yes,Yes,7.29,39.33,79.81
4,No,Yes,No,No,6.14,38.43,76.29
5,Yes,No,Yes,Yes,6.80,37.60,67.10
6,Yes,Yes,Yes,No,6.19,39.42,75.04
7,No,No,Yes,No,11.00,29.67,86.67
8,No,Yes,Yes,Yes,7.00,41.60,74.50
9,Yes,No,No,No,9.22,35.06,82.06



Data suggests that social support from partners, friends, and parents significantly influences students' mental health outcomes. A supportive network correlates with lower depression scores and higher social connectedness, while a lack of support appears to contribute to higher levels of depression and acculturative stress.

Impact of Professional Help: Investigating the role of professional or doctor support on overall mental health.

In [39]:
SELECT professional_bi,doctor_bi, 
       ROUND(AVG(todep), 2) AS avg_phq9, 
       ROUND(AVG(tosc), 2) AS avg_scs, 
       ROUND(AVG(toas), 2) AS avg_asiss
FROM students
WHERE inter_dom = 'Inter'
GROUP BY professional_bi,doctor_bi;


,professional_bi,doctor_bi,avg_phq9,avg_scs,avg_asiss
0,Yes,Yes,7.89,37.52,81.41
1,Yes,No,7.70,38.50,76.35
2,No,Yes,5.63,43.25,60.25
3,No,No,8.25,36.93,75.21


Data indicates that access to professional and medical support is associated with lower depression scores and can enhance social connectedness among students. The presence of these support systems appears to mitigate some mental health challenges, although some students still report significant acculturative stress regardless of the support available.

How Religious Support Affects Stress: Use the religion_bi column to see if religious support lowers acculturative stress.

In [31]:
SELECT religion_bi, 
       ROUND(AVG(todep), 2) AS avg_phq9, 
       ROUND(AVG(tosc), 2) AS avg_scs, 
       ROUND(AVG(toas), 2) AS avg_asiss
FROM students
WHERE inter_dom = 'Inter'
GROUP BY religion_bi;


,religion_bi,avg_phq9,avg_scs,avg_asiss
0,No,8.11,37.21,75.16
1,Yes,7.20,40.00,80.60


Data indicates that students with religious support tend to have lower depression scores and higher social connectedness compared to those without such support. However, the presence of religious support does not significantly reduce acculturative stress, indicating that other factors may contribute to stress levels for students adjusting to a new culture.

2. Impact of Being Alone on Mental Health
Since alone_bi could indicate whether a student feels alone or isolated, this can be particularly important to investigate for its impact on depression and social connectedness.

Query to Analyze Impact of Feeling Alone:

In [35]:
SELECT alone_bi, 
       ROUND(AVG(todep), 2) AS avg_phq9, 
       ROUND(AVG(tosc), 2) AS avg_scs, 
       ROUND(AVG(toas), 2) AS avg_asiss
FROM students
WHERE inter_dom = 'Inter'
GROUP BY alone_bi;


,alone_bi,avg_phq9,avg_scs,avg_asiss
0,No,7.08,39.49,73.12
1,Yes,10.60,31.91,82.05


Data indicates that students who feel alone experience higher levels of depression and lower social connectedness, highlighting the negative impact of loneliness on mental health. Additionally, the higher acculturative stress score among students who feel alone underscores the importance of social support in mitigating stress associated with cultural adjustment.

3. Role of Internet and Phone Use on Social Connectedness
You can explore whether internet or phone use impacts social connectedness and acculturative stress.

Query for Internet Use:

In [36]:
SELECT internet_bi, 
       ROUND(AVG(tosc), 2) AS avg_scs, 
       ROUND(AVG(toas), 2) AS avg_asiss
FROM students
WHERE inter_dom = 'Inter'
GROUP BY internet_bi;


,internet_bi,avg_scs,avg_asiss
0,No,37.60,74.93
1,Yes,36.54,78.57


Data suggests that students who do not use the internet may feel slightly more connected socially, while those who do use the internet experience higher levels of acculturative stress. This highlights the complexity of internet usage's impact on mental health, suggesting it may not always provide the support that students need during their adjustment to a new culture.

In [37]:
SELECT phone_bi, 
       ROUND(AVG(tosc), 2) AS avg_scs, 
       ROUND(AVG(toas), 2) AS avg_asiss
FROM students
WHERE inter_dom = 'Inter'
GROUP BY phone_bi;


,phone_bi,avg_scs,avg_asiss
0,No,37.28,75.23
1,Yes,38.57,78.38


Data indicates that having phone access is associated with higher social connectedness among students, suggesting that phones can serve as a valuable tool for maintaining relationships. However, the increase in acculturative stress scores among phone users highlights the potential challenges that come with connectivity in a new cultural environment.

CONCLUSION